In [3]:
import sys
import os


chemin_dossier = os.path.abspath('../scripts')
if chemin_dossier not in sys.path:
    sys.path.append(chemin_dossier)



In [5]:
from scryfall_api import *
from manage_data import *
import time

In [10]:
def get_random_card():
    random_card = []
    url_test = url_pytest
    response = r.get(url_test)
    data_random = response.json()
    print(data_random)

In [11]:
get_random_card()

{'object': 'card', 'id': 'efae4d84-8134-461a-a352-a5bdff7259a7', 'oracle_id': 'fe742976-ef13-4c25-972f-254b1ed436de', 'multiverse_ids': [464120], 'mtgo_id': 72714, 'tcgplayer_id': 190966, 'cardmarket_id': 374685, 'name': 'Mother Bear', 'lang': 'en', 'released_at': '2019-06-14', 'uri': 'https://api.scryfall.com/cards/efae4d84-8134-461a-a352-a5bdff7259a7', 'scryfall_uri': 'https://scryfall.com/card/mh1/171/mother-bear?utm_source=api', 'layout': 'normal', 'highres_image': True, 'image_status': 'highres_scan', 'image_uris': {'small': 'https://cards.scryfall.io/small/front/e/f/efae4d84-8134-461a-a352-a5bdff7259a7.jpg?1562202126', 'normal': 'https://cards.scryfall.io/normal/front/e/f/efae4d84-8134-461a-a352-a5bdff7259a7.jpg?1562202126', 'large': 'https://cards.scryfall.io/large/front/e/f/efae4d84-8134-461a-a352-a5bdff7259a7.jpg?1562202126', 'png': 'https://cards.scryfall.io/png/front/e/f/efae4d84-8134-461a-a352-a5bdff7259a7.png?1562202126', 'art_crop': 'https://cards.scryfall.io/art_crop/fro

In [3]:
def get_all_cards_today():
    all_cards = []
    url = url_en
    MAX_RETRIES = 10

    while url:
        for attempt in range(MAX_RETRIES):
            try:
                response = r.get(url, timeout=10)
                response.raise_for_status()
                data = response.json()

                # Extraire les objets 'card'
                all_cards.extend(data['data'])

                print(f"{len(all_cards)} cartes chargées...")
                url = data.get("next_page")
                time.sleep(0.2)  # Pause légère entre les requêtes

                break  # Sort du retry loop si succès

            except r.exceptions.RequestException as e:
                wait_time = (2 ** attempt)
                print(f"Erreur : {e}. Nouvelle tentative dans {wait_time} sec...")
                time.sleep(wait_time)
        else:
            print("Échec après plusieurs tentatives. Arrêt.")
            break

    # Transformer en DataFrame
    df = pd.json_normalize(all_cards)
    print(f"{len(df)} cartes importées.")
    return df


In [4]:
df = get_all_cards_today()

175 cartes chargées...
350 cartes chargées...
525 cartes chargées...
700 cartes chargées...
875 cartes chargées...
1050 cartes chargées...
1225 cartes chargées...
1400 cartes chargées...
1575 cartes chargées...
1750 cartes chargées...
1925 cartes chargées...
2100 cartes chargées...
2275 cartes chargées...
2450 cartes chargées...
2625 cartes chargées...
2800 cartes chargées...
2975 cartes chargées...
3150 cartes chargées...
3325 cartes chargées...
3500 cartes chargées...
3675 cartes chargées...
3850 cartes chargées...
4025 cartes chargées...
4200 cartes chargées...
4375 cartes chargées...
4550 cartes chargées...
4725 cartes chargées...
4900 cartes chargées...
5075 cartes chargées...
5250 cartes chargées...
5425 cartes chargées...
5600 cartes chargées...
5775 cartes chargées...
5950 cartes chargées...
6125 cartes chargées...
6300 cartes chargées...
6475 cartes chargées...
6650 cartes chargées...
6825 cartes chargées...
7000 cartes chargées...
7175 cartes chargées...
7350 cartes chargées.

In [5]:
dft = df.drop_duplicates(subset=["name"])


In [6]:
print(dft.columns.tolist())

['object', 'id', 'oracle_id', 'multiverse_ids', 'tcgplayer_id', 'cardmarket_id', 'name', 'lang', 'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image', 'image_status', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'power', 'toughness', 'colors', 'color_identity', 'keywords', 'games', 'reserved', 'game_changer', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'collector_number', 'digital', 'rarity', 'flavor_text', 'card_back_id', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame', 'full_art', 'textless', 'booster', 'story_spotlight', 'image_uris.small', 'image_uris.normal', 'image_uris.large', 'image_uris.png', 'image_uris.art_crop', 'image_uris.border_crop', 'legalities.standard', 'legalities.future', 'legalities.historic', 'legalities.timeless', 'legalities.gladiator', 'legalities.pioneer', 'legalities.mo

In [7]:
data_info = dft[[
        "id", "name", "lang","oracle_text", "set_name", "rarity",
        "released_at","image_uris.normal"
    ]].values.tolist()

In [10]:
print(data_info[0])

['5aa90ab6-2686-4462-8725-5d4370c05437', '_____', 'en', "{1}: This card's name becomes the card name of your choice. Activate anywhere, anytime.", 'Unhinged', 'uncommon', '2004-11-19', 'https://cards.scryfall.io/normal/front/5/a/5aa90ab6-2686-4462-8725-5d4370c05437.jpg?1663738897']


In [17]:
conn.rollback()

In [8]:
import psycopg2

def connect_database2():
    conn = psycopg2.connect(
        dbname="analyse_cartes",
        user="airflow",
        password="airflow",
        host="localhost",
        port="5433"
    )

    cur = conn.cursor()
    return conn, cur

In [9]:
conn, cur = connect_database2()

In [10]:
query_banlist = """
        INSERT INTO banlist
        VALUES (%s, %s, %s, %s, %s, %s)
    """

In [11]:
data_banlist = dft[[
        "name", "legalities.commander", "legalities.pauper", "legalities.standard", "legalities.modern", "legalities.vintage"
    ]].values.tolist()

In [12]:
print(data_banlist[10])

["Aang's Iceberg", 'not_legal', 'not_legal', 'not_legal', 'not_legal', 'not_legal']


In [18]:
cur.executemany(query_banlist, data_banlist)

In [19]:
conn.commit()

In [20]:
conn.close()

In [ ]:
url2 = "https://api.scryfall.com/cards/search?q=lang:en"

In [ ]:
liste_colonne=['id','oracle_id','name',"lang","set_name","rarity","released_at","prices.eur","prices.eur_foil"]
df_carte = df[liste_colonne]

In [ ]:
df_carte.tail()

In [ ]:
import requests as r
import pandas as pd
import time

all_cards = []

MAX_RETRIES = 10

while url:
    for attempt in range(MAX_RETRIES):
        try:
            response = r.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()

            # Extraire les objets 'card'
            all_cards.extend(data['data'])

            print(f"{len(all_cards)} cartes chargées...")
            url = data.get("next_page")
            time.sleep(0.2)  # Pause légère entre les requêtes

            break  # Sort du retry loop si succès

        except r.exceptions.RequestException as e:
            wait_time = (2 ** attempt)
            print(f"Erreur : {e}. Nouvelle tentative dans {wait_time} sec...")
            time.sleep(wait_time)
    else:
        print("Échec après plusieurs tentatives. Arrêt.")
        break

# Transformer en DataFrame
df = pd.json_normalize(all_cards)
print(f"{len(df)} cartes importées.")


In [ ]:
df.shape

In [ ]:
import psycopg2

conn = psycopg2.connect(
    dbname="prix_cartes",
    user="postgres",
    password="aE6C1U>4",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

In [ ]:
query_info = """
    INSERT INTO cartes (
        id, oracle_id, name, lang, set_name, rarity,
        released_at
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
data_info = df[[
    "id", "oracle_id", "name", "lang", "set_name", "rarity",
    "released_at"
]].values.tolist()



In [ ]:
data_info[0]

In [ ]:
cur.executemany(query_info, data_info)

conn.commit()

In [ ]:
liste_colonne=['id','oracle_id','name',"lang","set_name","rarity","released_at","prices.eur","prices.eur_foil"]
df_carte = df[liste_colonne]

In [ ]:
df_carte.head